In [ ]:
pip install asyncpraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
import asyncio
import asyncpraw
import pandas as pd
from datetime import datetime

async def fetch_comments(comment, comments_list, processed_ids, submission_id, indent=0):
    await comment.refresh()
    for reply in comment.replies:
        if reply.id in processed_ids:
            continue
        processed_ids.add(reply.id)

        comment_date = datetime.utcfromtimestamp(reply.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        comments_list.append({
            'submission_id': submission_id,
            'comment_id': reply.id,
            'parent_id': comment.id if indent > 0 else '',
            'body': reply.body,
            'author': str(reply.author),
            'created_utc': comment_date,
            'indent': indent
        })
        await fetch_comments(reply, comments_list, processed_ids, submission_id, indent=indent + 1)

async def main():
    reddit = asyncpraw.Reddit(
        client_id='26I6lu5xjL2TW6B1apvokA',
        client_secret='_nHSygU8rvx8J18Hp4eHH-PbN_jEvw',
        user_agent='SummativeAPI by WorryOk9238 (Contact: juliemorrisonjp@gmail.com)',
    )

    submission_id = '11ylnft'
    submission = await reddit.submission(id=submission_id)
    await submission.load()

    submission_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    await submission.comments.replace_more(limit=None)
    comments_list = []
    processed_ids = set()

    for comment in submission.comments.list():
        if comment.id in processed_ids:
            continue
        processed_ids.add(comment.id)

        comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        comments_list.append({
            'submission_id': submission_id,
            'comment_id': comment.id,
            'parent_id': '',
            'body': comment.body,
            'author': str(comment.author),
            'created_utc': comment_date,
            'indent': 0
        })
        await fetch_comments(comment, comments_list, processed_ids, submission_id)

    df = pd.DataFrame(comments_list)

    await reddit.close()

    return df

async def run():
    df = await main()
    df.to_csv('4.csv', index=False)
    print(df)

await run()

   submission_id comment_id parent_id  \
0        11ylnft    jd8904k             
1        11ylnft    jd8c55n             
2        11ylnft    jdb38a3   jd8c55n   
3        11ylnft    juymrnf   jd8c55n   
4        11ylnft    juytvhn   juymrnf   
5        11ylnft    jdb2wuz             
6        11ylnft    jd8s0fg             
7        11ylnft    jdbd52y             
8        11ylnft    jji7vyk             
9        11ylnft    jd8wtm1             
10       11ylnft    jdb5ciq             
11       11ylnft    jd94zs2             
12       11ylnft    jdb5upa             
13       11ylnft    jlz16nu   jdb5upa   
14       11ylnft    jdbdfnr             
15       11ylnft    jda8i5r             
16       11ylnft    jdb6bup             
17       11ylnft    jdbbzm1             
18       11ylnft    jdc57za   jdbbzm1   
19       11ylnft    jd8w7nk             
20       11ylnft    jd9fpre             
21       11ylnft    jdbcfvt             
22       11ylnft    jdbdiv3             
23       11ylnft

In [ ]:
csv_files = ['1.csv', '2.csv', '3.csv', '4.csv']

dfs = [pd.read_csv(file) for file in csv_files]

combined_df = pd.concat(dfs, ignore_index=True)

combined_df.to_csv('reddit_netnography_comments.csv', index=False)
